In this notebook, we introduce the augmented Lagrangian method and its variants. Let us start with the optimization with equaltiy constraints
$$
\begin{equation}
\begin{aligned}
\min_x\; &\frac{1}{2} x^\top Q x + c^\top x \\
\text{s.t.}\; & Ax = b,
\end{aligned}
\end{equation}
$$
where $x \in \mathbb{R}^{n}$, $Q \in \mathbb{R}^{n\times n}$, $c \in \mathbb{R}^{n}$, $A \in \mathbb{R}^{m\times n}$, $b \in \mathbb{R}^{m}$. Equivalently, we write its min-max formulation as 
$$
\min_x \max_{\lambda}\; \frac{1}{2} x^\top Q x + c^\top x + \lambda^\top (Ax - b),
$$
where $\lambda \in \mathbb{R}^{n}$ is the Lagrange multiplier vector related to the equality constraints. 
For readers who are not familiar with  the min-max formulation, please refer to [the video](https://www.youtube.com/watch?v=uh1Dk68cfWs).
The corresponding KKT conditions is then given by
$$
\begin{bmatrix}
Q & A^\top \\
A & 0
\end{bmatrix}
\begin{bmatrix}
x \\ \lambda
\end{bmatrix} = 
\begin{bmatrix}
-c \\ b
\end{bmatrix}.
$$
Note that the KKT matrix is nonsingular if the following assumptions holds.
- The constraint Jacobian $A$ has **full row rank**;
- The matrix $Q$ is positive definite on the **tangent space** of the constraints, that is, $d^\top Q d > 0$ for all $d \neq 0$ such that $Ad = 0$.

The first assumption implies that there are no redundant constraints. 
The second assumption indicates that the objective function is strictly convex in all feasible directions.
Here, we introduce one method for addressing situations in which the first assumption is violated.
When the row rank of $A_k$ is deficient, the row vectors of $A_k$ does not span the full space of constraints. Consequently, the dual variable $\lambda$ may not be uniquely determined, and the min-max problem and the KKT system above may not be well-posed. To address this, we add a regularization term related to $\lambda$, yielding
$$
\min_x \max_{\lambda}\; \frac{1}{2} x^\top Q x + c^\top x + \lambda^\top (Ax - b) - \frac{\rho}{2}\Vert \lambda - \bar{\lambda}\Vert_2^2.
$$
A typical choice of $\bar{\lambda}$ is the multiplier vector obtained from the last iteration. The modified KKT system is then
$$
\begin{equation}
\begin{bmatrix}
Q & A^\top \\
A & -\rho I 
\end{bmatrix}
\begin{bmatrix}
x \\ \lambda
\end{bmatrix} = 
\begin{bmatrix}
-c \\ b - \rho\bar{\lambda}
\end{bmatrix}.
\end{equation} \tag{2}
$$
To obtain the solution to the optimization problem in (1), we need to solve a sequence of KKT systems in (2) using the $LDL^\top$ factorization. Alternatively, in the min-max formulation, by solving the inner maximization problem, we express $\lambda$ as
$$
\lambda = \frac{1}{\rho}(Ax - b) + \bar{\lambda}.
$$
Substituting $\lambda$ into the outer minimization problem, we obtain
$$
\begin{aligned}
 & \min_ x\; \frac{1}{2} x^\top Q x + c^\top x + \left( \frac{1}{\rho}(Ax - b) + \bar{\lambda} \right)^\top (Ax - b) - \frac{\rho}{2} \left\Vert \frac{1}{\rho} (Ax - b) \right\Vert_2^2 \\
= & \min_ x\; \underbrace{\frac{1}{2} x^\top Q x + c^\top x + \bar{\lambda}^\top (Ax - b) + \frac{1}{2\rho} \Vert Ax-b \Vert_2^2,}_{\mathcal{L}_{a}(x, \bar{\lambda}; \rho)}
\end{aligned}
$$
where $\mathcal{L}_{a}$ is known as the augmented Lagrangian function. Equivalently, by completing the square, we can rewrite $\mathcal{L}_{a}$ as 
$$
\mathcal{L}_{a}(x, \bar{\lambda}; \rho) := \frac{1}{2} x^\top Q x + c^\top x + \frac{1}{2\rho}\left\Vert Ax -b + \rho\bar{\lambda} \right\Vert_2^2 - \frac{\rho}{2} \Vert \bar{\lambda} \Vert^2_2.
$$
The augmented Lagrangian method (ALM) alternates between the minimization of $\mathcal{L}_{a}$ with respect to the primal variables $x$ and a simple update rule for the dual variables $\lambda$:
$$
\begin{aligned}
x^{k+1} &= \text{arg}\min_x \mathcal{L}_{a}(x, \lambda^k; \rho^k) \\
\lambda^{k+1} &= \lambda^k + \frac{1}{\rho^k}(Ax^{k+1} - b),
\end{aligned}
$$
where $k$ represents the iteration number.

<!-- We consider the following optimization problem
$$
\begin{aligned}
    \min_{x}\; &f(x) \\
    \text{s.t.}\; &h(x) \leq 0.
\end{aligned}
$$
By introducing the slack vector $s$, we rewrite the optimization problem as
$$
\begin{aligned}
    \min_{x, s}\; &f(x) \\
    \text{s.t.}\; & \underbrace{h(x) + [s]^2}_{g(x, s)} = 0,
\end{aligned}
$$
where $[\cdot]^2$ represents element-wise squaring. To derive the augmented Lagrangian (AL) method, we consider the min-max formulation
$$
\min_{x, s} \max_{\lambda}\; f(x) + \lambda^\top g(x, s),
$$
where $\lambda$ is the Lagrange multiplier vector. 

$$
\min_{x, s} \max_{\lambda}\; f(x) + \lambda^\top g(x, s) - \frac{1}{2} \Vert \lambda - \bar{\lambda} \Vert^2
$$
We can now solve the inner maximization problem
$$
\lambda^*(\bar{\lambda}) = \bar{\lambda} + \rho g(x, s).
$$

$$
\begin{aligned}
&\min_{x, s} \max_{\lambda}\; f(x) + \lambda^\top g(x, s) - \frac{1}{2} \Vert \lambda - \bar{\lambda} \Vert^2 \\
= &\min_{x, s}\; f(x) + \lambda^*(\bar{\lambda})^\top g(x, s) - \frac{1}{2} \Vert \lambda^*(\bar{\lambda}) - \bar{\lambda} \Vert^2 \\
= &\min_{x, s}\; f(x) + \left(\bar{\lambda} + \rho g(x, s)\right)^\top g(x, s) - \frac{\rho}{2} \left\Vert g(x, s) \right\Vert^2 \\
= &\min_{x, s}\; f(x) + \bar{\lambda}^\top g(x, s) + \frac{\rho}{2} \left\Vert g(x, s) \right\Vert^2 \\
= &\min_{x, s}\; f(x) + \frac{\rho}{2} \left\Vert g(x, s) + \frac{\bar{\lambda}}{\rho} \right\Vert^2 - \frac{1}{2\rho} \Vert \bar{\lambda} \Vert^2 \\
= &\min_{x} \min_{s} f(x) + \frac{\rho}{2} \left\Vert h(x) + [s]^2 + \frac{\bar{\lambda}}{\rho} \right\Vert^2 - \frac{1}{2\rho} \Vert \bar{\lambda} \Vert^2 \\
= &\min_{x}\; f(x) + \frac{\rho}{2} \left\Vert \max\left( h(x) + \frac{\lambda}{\rho}, 0 \right) \right\Vert^2 - \frac{1}{2\rho} \Vert \bar{\lambda} \Vert^2
\end{aligned}
$$

$$
s^{*2}_i = 
\begin{cases}
    0\quad &\text{if}\;  g_i(x) + \frac{\bar{\lambda}}{\rho} < 0\\
    -g_i(x) - \frac{\bar{\lambda}}{\rho}\quad &\text{if}\;  g_i(x) + \frac{\bar{\lambda}}{\rho} \geq 0
\end{cases}
$$ -->

Next, we consider the quadratic program with inequality constraints
$$
\begin{aligned}
\min_x\; &\frac{1}{2} x^\top Q x + c^\top x \\
\text{s.t.}\; & Cx \leq d.
\end{aligned}
$$
By introducing a slack vector $s$, we can rewrite the optimization problem as
$$
\begin{aligned}
\min_{x, s}\; &\frac{1}{2} x^\top Q x + c^\top x \\
\text{s.t.}\; & Cx + s = d \\
& s \ge 0.
\end{aligned}
$$
As above, we consider the regularized min-max formulation
$$
\min_{x,\, s \ge 0} \max_{y}\; \frac{1}{2} x^\top Q x + c^\top x + y^\top (Cx - d + s) - \frac{\mu}{2}\Vert y - \bar{y}\Vert_2^2.
$$
Solving the inner optimization problem, we obtain
$$
\begin{equation}
\min_{x,\, s \ge 0}\; \frac{1}{2} x^\top Q x + c^\top x + \frac{1}{2\mu}\left\Vert Cx - d + s + \mu\bar{y} \right\Vert_2^2 - \frac{\mu}{2} \Vert \bar{y} \Vert^2_2.
\end{equation} \tag{3}
$$
Next, we minimize the objective function above over the slack vector $s$, yielding
$$
s^* = 
\begin{cases}
- Cx + d - \mu\bar{y}\quad &\text{if}\; Cx - d \leq -\mu\bar{y}, \\
0 & \text{otherwise}.
\end{cases}
$$

Substituting $s^*$ into (3), we obtain
$$
\min_{x}\; \underbrace{\frac{1}{2} x^\top Q x + c^\top x + \frac{1}{2\mu}\left\Vert \max(Cx - d + \mu\bar{y}, 0) \right\Vert_2^2 - \frac{\mu}{2} \Vert \bar{y} \Vert^2_2}_{\mathcal{L}_{a}(x, \bar{y};\, \mu)}.
$$

The update rule is then given by
$$
\begin{aligned}
x^{k+1} &= \text{arg}\min_x \mathcal{L}_{a}(x, y^k; \mu^k) \\ 
y^{k+1} &= y^k + \frac{1}{\mu^k}(Cx^{k+1} - d + s^*) \\
&= \max \left( y^k + \frac{1}{\mu^k}\left(Cx^{k+1} - d\right), 0 \right).
\end{aligned}
$$